# Lab 4.1.4: Document AI Pipeline - SOLUTIONS

This notebook contains the complete solution for the Invoice Processing System challenge.

---

In [ ]:
# Setup
import gc
import re
import json
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
from pathlib import Path

import torch
from PIL import Image
import fitz

from transformers import AutoProcessor, LlavaForConditionalGeneration

# Load VLM
print("Loading LLaVA for invoice processing...")
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = LlavaForConditionalGeneration.from_pretrained(
    "llava-hf/llava-1.5-7b-hf",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

print("✅ Ready!")

---

## Challenge Solution: Invoice Processing System

In [ ]:
@dataclass
class LineItem:
    """A single line item from an invoice."""
    description: str
    quantity: Optional[float] = None
    unit_price: Optional[float] = None
    amount: Optional[float] = None


@dataclass
class InvoiceData:
    """Structured invoice data."""
    invoice_number: Optional[str] = None
    date: Optional[str] = None
    due_date: Optional[str] = None
    vendor_name: Optional[str] = None
    vendor_address: Optional[str] = None
    customer_name: Optional[str] = None
    customer_address: Optional[str] = None
    line_items: List[LineItem] = None
    subtotal: Optional[float] = None
    tax: Optional[float] = None
    total: Optional[float] = None
    currency: str = "USD"
    confidence: float = 0.0
    validation_errors: List[str] = None
    
    def __post_init__(self):
        if self.line_items is None:
            self.line_items = []
        if self.validation_errors is None:
            self.validation_errors = []
    
    def to_dict(self) -> Dict:
        """Convert to dictionary."""
        result = {
            "invoice_number": self.invoice_number,
            "date": self.date,
            "due_date": self.due_date,
            "vendor": {
                "name": self.vendor_name,
                "address": self.vendor_address,
            },
            "customer": {
                "name": self.customer_name,
                "address": self.customer_address,
            },
            "line_items": [
                asdict(item) for item in self.line_items
            ],
            "subtotal": self.subtotal,
            "tax": self.tax,
            "total": self.total,
            "currency": self.currency,
            "confidence": self.confidence,
            "validation_errors": self.validation_errors,
        }
        return result
    
    def to_json(self, indent: int = 2) -> str:
        """Convert to JSON string."""
        return json.dumps(self.to_dict(), indent=indent)

In [ ]:
def analyze_invoice_image(image: Image.Image, question: str, max_tokens: int = 512) -> str:
    """Get VLM analysis of invoice image."""
    prompt = f"USER: <image>\n{question}\nASSISTANT:"
    
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
        )
    
    response = processor.decode(output_ids[0], skip_special_tokens=True)
    if "ASSISTANT:" in response:
        response = response.split("ASSISTANT:")[-1].strip()
    
    return response


def parse_currency(text: str) -> Optional[float]:
    """Extract currency value from text."""
    # Remove currency symbols and commas
    cleaned = re.sub(r'[£€$,]', '', text)
    # Find numbers
    match = re.search(r'[\d]+\.?[\d]*', cleaned)
    if match:
        try:
            return float(match.group())
        except ValueError:
            return None
    return None


def validate_invoice(invoice: InvoiceData) -> List[str]:
    """Validate extracted invoice data."""
    errors = []
    
    # Check required fields
    if not invoice.invoice_number:
        errors.append("Missing invoice number")
    if not invoice.date:
        errors.append("Missing invoice date")
    if not invoice.vendor_name:
        errors.append("Missing vendor name")
    if not invoice.total:
        errors.append("Missing total amount")
    
    # Validate line items sum to total
    if invoice.line_items and invoice.total:
        items_sum = sum(
            item.amount for item in invoice.line_items
            if item.amount is not None
        )
        
        if invoice.subtotal:
            if abs(items_sum - invoice.subtotal) > 0.01:
                errors.append(f"Line items sum ({items_sum:.2f}) doesn't match subtotal ({invoice.subtotal:.2f})")
        
        expected_total = items_sum + (invoice.tax or 0)
        if abs(expected_total - invoice.total) > 0.01:
            errors.append(f"Calculated total ({expected_total:.2f}) doesn't match stated total ({invoice.total:.2f})")
    
    return errors


def process_invoice(invoice_path: str) -> Dict[str, Any]:
    """
    Process an invoice and extract structured data.
    
    Args:
        invoice_path: Path to invoice PDF or image
        
    Returns:
        Dictionary with extracted invoice data
    """
    path = Path(invoice_path)
    
    # Load image
    if path.suffix.lower() == '.pdf':
        # Convert PDF to image
        doc = fitz.open(invoice_path)
        page = doc[0]
        mat = fitz.Matrix(2, 2)  # 2x zoom for quality
        pix = page.get_pixmap(matrix=mat)
        image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        doc.close()
    else:
        image = Image.open(invoice_path).convert("RGB")
    
    print(f"📄 Processing invoice: {path.name}")
    
    invoice = InvoiceData()
    
    # Extract invoice number
    print("  Extracting invoice number...")
    response = analyze_invoice_image(
        image,
        "What is the invoice number on this invoice? Just give me the number.",
        max_tokens=50
    )
    invoice.invoice_number = response.strip()
    
    # Extract date
    print("  Extracting date...")
    response = analyze_invoice_image(
        image,
        "What is the invoice date? Give me just the date.",
        max_tokens=50
    )
    invoice.date = response.strip()
    
    # Extract vendor
    print("  Extracting vendor info...")
    response = analyze_invoice_image(
        image,
        "Who is the vendor/seller on this invoice? Give me the company name.",
        max_tokens=100
    )
    invoice.vendor_name = response.strip()
    
    # Extract line items
    print("  Extracting line items...")
    response = analyze_invoice_image(
        image,
        """List all the line items on this invoice. For each item, provide:
        - Description
        - Quantity (if shown)
        - Unit price (if shown)
        - Amount/Total
        Format: Description | Quantity | Unit Price | Amount""",
        max_tokens=500
    )
    
    # Parse line items (basic parsing - would need more robust parsing for production)
    for line in response.split('\n'):
        if '|' in line:
            parts = [p.strip() for p in line.split('|')]
            if len(parts) >= 2:
                item = LineItem(
                    description=parts[0],
                    quantity=parse_currency(parts[1]) if len(parts) > 1 else None,
                    unit_price=parse_currency(parts[2]) if len(parts) > 2 else None,
                    amount=parse_currency(parts[-1]),
                )
                invoice.line_items.append(item)
    
    # Extract totals
    print("  Extracting totals...")
    response = analyze_invoice_image(
        image,
        "What is the total amount on this invoice? Just give me the number.",
        max_tokens=50
    )
    invoice.total = parse_currency(response)
    
    response = analyze_invoice_image(
        image,
        "What is the tax amount on this invoice? Just give me the number, or 'none' if no tax.",
        max_tokens=50
    )
    if 'none' not in response.lower():
        invoice.tax = parse_currency(response)
    
    # Validate
    print("  Validating...")
    invoice.validation_errors = validate_invoice(invoice)
    
    # Calculate confidence based on filled fields and validation
    filled_fields = sum([
        invoice.invoice_number is not None,
        invoice.date is not None,
        invoice.vendor_name is not None,
        invoice.total is not None,
        len(invoice.line_items) > 0,
    ])
    base_confidence = filled_fields / 5
    validation_penalty = len(invoice.validation_errors) * 0.1
    invoice.confidence = max(0, min(1, base_confidence - validation_penalty))
    
    print(f"\n✅ Processing complete!")
    print(f"   Confidence: {invoice.confidence:.0%}")
    if invoice.validation_errors:
        print(f"   ⚠️ Validation errors: {len(invoice.validation_errors)}")
    
    return invoice.to_dict()

In [ ]:
# Create a sample invoice image for testing
def create_sample_invoice() -> Image.Image:
    """Create a simple sample invoice image."""
    from PIL import ImageDraw, ImageFont
    
    # Create blank image
    img = Image.new('RGB', (800, 1000), 'white')
    draw = ImageDraw.Draw(img)
    
    try:
        font_large = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 24)
        font_medium = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 16)
        font_small = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 14)
    except:
        font_large = font_medium = font_small = ImageFont.load_default()
    
    # Header
    draw.text((50, 30), "ACME Corporation", fill='black', font=font_large)
    draw.text((50, 60), "123 Business St, Tech City, TC 12345", fill='gray', font=font_small)
    
    # Invoice info
    draw.text((600, 30), "INVOICE", fill='darkblue', font=font_large)
    draw.text((550, 70), "Invoice #: INV-2024-0042", fill='black', font=font_medium)
    draw.text((550, 95), "Date: December 15, 2024", fill='black', font=font_medium)
    draw.text((550, 120), "Due: January 15, 2025", fill='black', font=font_medium)
    
    # Bill to
    draw.text((50, 150), "Bill To:", fill='gray', font=font_small)
    draw.text((50, 170), "Tech Solutions Inc.", fill='black', font=font_medium)
    draw.text((50, 195), "456 Client Ave, Business Park", fill='black', font=font_small)
    
    # Table header
    y = 280
    draw.rectangle([(40, y-5), (760, y+25)], fill='lightgray')
    draw.text((50, y), "Description", fill='black', font=font_medium)
    draw.text((400, y), "Qty", fill='black', font=font_medium)
    draw.text((500, y), "Unit Price", fill='black', font=font_medium)
    draw.text((650, y), "Amount", fill='black', font=font_medium)
    
    # Line items
    items = [
        ("Software Development Services", 40, 150.00, 6000.00),
        ("Cloud Hosting (Monthly)", 1, 299.00, 299.00),
        ("Technical Support", 10, 75.00, 750.00),
        ("Documentation", 1, 500.00, 500.00),
    ]
    
    y = 320
    for desc, qty, price, amount in items:
        draw.text((50, y), desc, fill='black', font=font_small)
        draw.text((400, y), str(qty), fill='black', font=font_small)
        draw.text((500, y), f"${price:.2f}", fill='black', font=font_small)
        draw.text((650, y), f"${amount:.2f}", fill='black', font=font_small)
        y += 35
    
    # Totals
    y = 520
    draw.line([(500, y), (760, y)], fill='gray')
    draw.text((500, y+10), "Subtotal:", fill='black', font=font_medium)
    draw.text((650, y+10), "$7,549.00", fill='black', font=font_medium)
    
    draw.text((500, y+40), "Tax (8%):", fill='black', font=font_medium)
    draw.text((650, y+40), "$603.92", fill='black', font=font_medium)
    
    draw.line([(500, y+70), (760, y+70)], fill='black', width=2)
    draw.text((500, y+80), "TOTAL:", fill='black', font=font_large)
    draw.text((640, y+80), "$8,152.92", fill='darkblue', font=font_large)
    
    # Footer
    draw.text((50, 900), "Payment Terms: Net 30", fill='gray', font=font_small)
    draw.text((50, 920), "Thank you for your business!", fill='gray', font=font_small)
    
    return img

# Create and display sample invoice
sample_invoice = create_sample_invoice()

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 12))
plt.imshow(sample_invoice)
plt.axis('off')
plt.title("Sample Invoice")
plt.tight_layout()
plt.show()

# Save for processing
sample_invoice.save("sample_invoice.png")

In [ ]:
# Process the invoice
result = process_invoice("sample_invoice.png")

print("\n📊 Extracted Invoice Data:")
print("=" * 60)
print(json.dumps(result, indent=2))

---

## Cleanup

In [ ]:
import os
os.remove("sample_invoice.png")

del model, processor
torch.cuda.empty_cache()
gc.collect()
print("✅ Cleanup complete!")